# 🧩 Módulo 7 — Pandas: Agrupaciones y `groupby`

En este notebook aprenderás a hacer análisis avanzado con `groupby`, incluyendo:

- Agrupaciones básicas
- Múltiples agregaciones
- Transformaciones
- Filtrado de grupos
- Tablas dinámicas (`pivot_table`)
- Multiíndice

---

## 1️⃣ Dataset de ejemplo

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'categoria': ['A','A','B','B','B','C','C'],
    'producto': ['p1','p2','p3','p4','p5','p6','p7'],
    'ventas': [100, 150, 200, 50, 120, 80, 60],
    'coste': [60, 90, 120, 30, 70, 40, 20]
})
df

---
## 2️⃣ Agrupaciones básicas

Ventas totales por categoría:

In [ ]:
df.groupby('categoria')['ventas'].sum()

Promedio de ventas por categoría:

In [ ]:
df.groupby('categoria')['ventas'].mean()

---
## 3️⃣ Agrupaciones múltiples

Obtener estadísticas combinadas:

In [ ]:
df.groupby('categoria').agg({'ventas':['sum','mean'], 'coste':['sum','mean']})

Resumen estadístico por grupo:

In [ ]:
df.groupby('categoria')['ventas'].describe()

---
## 4️⃣ Transformaciones (`transform`)

Calcular ventas normalizadas por categoría:

In [ ]:
df['ventas_norm'] = df.groupby('categoria')['ventas'].transform(lambda x: (x - x.mean())/x.std())
df[['categoria','producto','ventas','ventas_norm']]

---
## 5️⃣ Filtrado de grupos (`filter`)

Ejemplo: seleccionar solo categorías con ventas totales ≥ 200:

In [ ]:
df.groupby('categoria').filter(lambda g: g['ventas'].sum() >= 200)

---
## 6️⃣ Tablas dinámicas (pivot table)


In [ ]:
pd.pivot_table(df, values='ventas', index='categoria', aggfunc='sum')

Pivot table con varias funciones:

In [ ]:
pd.pivot_table(df, values=['ventas','coste'], index='categoria', aggfunc=['sum','mean'])

---
## 7️⃣ multi-índice

Agrupar por categoría y producto:

In [ ]:
multi = df.groupby(['categoria','producto'])['ventas'].sum()
multi

Convertir multi-índice en columnas (reset index):

In [ ]:
multi.reset_index()

---
## 8️⃣ Ejercicio práctico

Usando `df`:

### 🧩 Objetivos
1. Calcula ventas totales por categoría
2. Calcula el margen como `ventas - coste` y obtén media por categoría
3. Normaliza `coste` por categoría usando `transform`
4. Haz un `filter` para quedarte solo con categorías cuyo *promedio de ventas* > 100
5. Construye una `pivot_table` que muestre el total de ventas y coste por categoría

Escribe tu solución aquí:

In [ ]:
# Tu solución aquí


---
## ✅ Solución (oculta)
<details>
<summary>Mostrar soluciones</summary>

```python
df.groupby('categoria')['ventas'].sum()
```

```python
df['margen'] = df['ventas'] - df['coste']
df.groupby('categoria')['margen'].mean()
```

```python
df['coste_norm'] = df.groupby('categoria')['coste'].transform(lambda x: (x-x.mean())/x.std())
```

```python
df.groupby('categoria').filter(lambda g: g['ventas'].mean() > 100)
```

```python
pd.pivot_table(df, values=['ventas','coste'], index='categoria', aggfunc='sum')
```
</details>